This project applies an additive attention model to a dataset of 100 customer feedbacks. Our task is to do sentiment analysis based on a small dataset and measure the performance of the model. First the word tokens in the input sequence are mapped to one-hot vectors; the performance of the model does not surpass 36% on the testing dataset which contains 25 samples. One our next trial, we map each token to GloVe word embeddings; training on the same model structure yields 96% accuracy both for the training and testing dataset. 

In [44]:
# Load packages: 
import pandas as pd 
from sklearn.model_selection import train_test_split 

from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply, Softmax
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import softmax
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K
import tensorflow as tf

import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer,LancasterStemmer
import re

In [3]:
def pred(y): 
    '''
    This function maps the probabilities outputed by the model back to the rankings list 
    and outputs the ranking with the highest probability. 
    
    inputs: 
    y  (1,m)     : Probability output of the RNN model 
    
    outputs: 
    res (string) : The ranking corresponding to the most probable outcome. 
    
    '''
    y = list(y)
    #ranking = ['Below Average' , 'Average' , 'Above Average']
    res = ranking[y.index(max(y))]
    return(res)


In [4]:
#need to write another function that maps the correct output of the function to the rankings. 
def vec_output(y): 
    """
    This function takes the y_test dataset and returns a one-hot vector for each sample. 
    
    """
    m = len(ranking)
    txt = y
    v = np.zeros(m) 
    j = ranking.index(txt)
    v[j] = 1
    return v 


In [5]:
def vec_input(x): 
    
    """
    This function takes any input (a sentence from customers), x, and returns a one-hot vector based on 
    words introduced in the vocabulary. This function returns k vectors where k is the number of words in the 
    sentence. Every vector corresponds to a word in the dictionary and has entries = 0 except the entry that 
    corresponds to the word in the dictionary.
    
    inputs: 
    
    x (string) : a statement from customers. 
    
    outputs: 
    v (m,n)    : where m is the number of words in the sentence and n is the number of total words in the dictionary. 
    
    """
    m = len(dictionary)
    txt = X_train[0]
    txt = txt.lower()
    txt = txt.split()
    txt = (txt[:10] if len(txt) > 10 else txt + ['<pad>'] * (10 - len(txt)))
    n = len(txt)
    v = np.zeros((n, m))
    for i in range(0, n): 
        j = dictionary.index(txt[i])
        v[i,j] = 1
        
    return(v)

In [6]:
#Loading the data: 
CustomerFeed = 'CustomerFeedback.xlsx'
df = pd.read_excel(CustomerFeed)
print(df)

                                            Sentence         Ranking 
0   looks beautiful I am in love with this product .   Above Average 
1                               I really like this .   Above Average 
2         I like this but the design could be better         Average 
3                            I do not like the smell   Below Average 
4           Works well but the smell is too strong .         Average 
..                                                ...             ...
95                                     not satisfied   Below Average 
96                                     does not work   Below Average 
97                               does not smell good   Below Average 
98                          does not work for my son   Below Average 
99                                     Saves me time   Above Average 

[100 rows x 2 columns]


In [7]:
dffed = df.iloc[:,0]
x = dffed.to_numpy()
dfrank = df.iloc[:,1]

y = dfrank.to_numpy()
print(x[1:5])
print(y[:3])

['I really like this . ' 'I like this but the design could be better '
 'I do not like the smell ' 'Works well but the smell is too strong . ']
['Above Average ' 'Above Average ' 'Average ']


In [8]:
ranking = np.unique(y)
ranking = ranking.tolist()
ranking

['Above Average ', 'Average ', 'Below Average ']

In [9]:
lenx = len(x)
Split = [] 
Dic = []
for i in range(0,len(x)):
    split = x[i].split()
    for i in range(0,len(split)): 
        split[i] = split[i].lower()
    Dic.extend(np.unique(split))
dictionary = np.unique(Dic)#this is our new dictionary. 
dictionary = dictionary.tolist()

# Add the extra padding token: 
dictionary =  dictionary + ["<pad>"] 

# Print the resutls: 
dictionary[1:10]

['a', 'about', 'after', 'am', 'amazing', 'and', 'average', 'bad', 'be']

In [10]:
#dividing the dataset into 75% training set and 25% test set: 
X_train, X_test, y_train, y_test = train_test_split(x,y, 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True) 
print(X_train[1:5])
print(y_train[1:5])

['looks amazing ' 'love this but design could be better '
 'I love the color ' 'good quality ']
['Above Average ' 'Average ' 'Above Average ' 'Average ']


In [11]:
X_train1 = np.array([vec_input(x) for x in X_train])
len(X_train1[3])


10

In [12]:
X_train1[0] #one hot encoding for the first element. 

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      

In [13]:
X_test1 = np.array([vec_input(x) for x in X_test])
y_train1 = (np.array([vec_output(y) for y in y_train])).reshape(len(y_train), 1, len(ranking))
y_test1 = (np.array([vec_output(y) for y in y_test])).reshape(len(y_test), 1, len(ranking))

In [14]:
y_train1.shape

(75, 1, 3)

In [15]:
y_test1.shape

(25, 1, 3)

In [16]:
y_train1[0:3]

array([[[1., 0., 0.]],

       [[1., 0., 0.]],

       [[0., 1., 0.]]])

In [17]:
print(f"Input sentence: {X_train[0]}\n")
print(f"One-hot vector representation of input:\n{X_train1[0]}\n")
print(f"Rating to be predicted: { y_train[0]}\n")
print(f"One-hot encoding for the output: {y_train1[0]}") 
print(ranking)

Input sentence: saves me time 

One-hot vector representation of input:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [18]:
X_train1 = np.array(X_train1)

In [19]:
X_train1.shape

(75, 10, 78)

In [20]:
len(dictionary)

78

In [21]:
def NeuralAttention(a,s_prev): 
    """
    Implements one step of attention mechanism
    
    Arguments:
    a -- output of the Bi-LSTM of shape (m, Tx, 2* n_a)  #(#samples, #rows, #columns)
    s_prev -- previous hidden state of the LSTM of shape (m, n_s)
    Tx -- length of the input sequence (Global Variable)

    Returns:
    context -- context vector, input of the next LSTM cell
    """
    #Create copies of s_prev 
    s_prev = RepeatVector(Tx)(s_prev) #what about all samples together 
    
    #Concatenate s_prev and a: 
    concat = Concatenate(axis = -1)([a,s_prev])
    
    #Run through the first layer of FFN with activation tanh and with 10 neurons: 
    dense1 = Dense(10, activation = "tanh")(concat) #[m,30,10+len(s_prev]-> [m,1,30]
    
    #Run through the final layer of FFN with activation ReLU and 1 neuron: 
    energies = Dense(1,activation = "relu")(dense1)
    
    #Run through a Softmax function to find alphas: 
    alphas = Softmax(axis = 1)(energies)
    
    #Multiply the alphas with their respective a<t'>: 
    Context = Dot(axes=1)([alphas,a])
    
    return(Context)

In [22]:
n_a = 10 # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
n_s = 30 # number of units for the post-attention LSTM's hidden state "s"

post_activation_LSTM_cell = LSTM(n_s, return_state = True) 
#output_layer = Dense(len(ranking), activation=Softmax)

In [75]:
def modelf(Tx, Ty, n_a, n_s, len_dictionary, ranking):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """

    
    # Define the inputs of your model with a shape (Tx, human_vocab_size)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx, len_dictionary))
    # initial hidden state
    s0 = Input(shape=(n_s,), name='s0')
    # initial cell state
    c0 = Input(shape=(n_s,), name='c0')
    # hidden state
    s = s0
    # cell state
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    # Define your pre-attention Bi-LSTM. a is a list of all the hidden states. 
    a = Bidirectional(LSTM(units=n_a, return_sequences=True))(X)
    
    # Perform one step of the attention mechanism to get back the context vector at step t 
    context = NeuralAttention(a,s)
        
    # Apply the post-attention LSTM cell to the "context" vector while also inputting the previous hidden state and cell state. 
    s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
       
    # Apply Dense layer to the hidden state at the last time step of post-attention LSTM 
    out = Dense(len(ranking),activation = "tanh")(s)
    # Run through a Softmax function: 
    res = Softmax(axis = 1)(out)
    # Append "out" to the "outputs" list 
    outputs.append(res)
    
    # Create model instance taking three inputs and returning the list of outputs.
    model = Model(inputs = [X,s0,c0], outputs = outputs)
    
    return model

In [76]:
Tx = 10
Ty = 1
n_a = 10
n_s = 30
len_dictionary = 78

In [77]:
model = modelf(Tx, Ty, n_a, n_s, len_dictionary, ranking)
model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 10, 78)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s0 (InputLayer)     │ (None, 30)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_3     │ (None, 10, 20)    │      7,120 │ input_layer_3[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_3     │ (None, 10, 30)    │          0 │ s0[0][0]          │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 10, 50)    │          0 │ bidirectional_3[… │
│ (Concatenate)       │                   │            │ repeat_vector_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 10, 10)    │        510 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 10, 1)     │         11 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_6 (Softmax) │ (None, 10, 1)     │          0 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_3 (Dot)         │ (None, 1, 20)     │          0 │ softmax_6[0][0],  │
│                     │                   │            │ bidirectional_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ c0 (InputLayer)     │ (None, 30)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 30),      │      6,120 │ dot_3[0][0],      │
│                     │ (None, 30),       │            │ s0[0][0],         │
│                     │ (None, 30)]       │            │ c0[0][0]          │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 3)         │         93 │ lstm[3][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_7 (Softmax) │ (None, 3)         │          0 │ dense_11[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,854 (54.12 KB)

 Trainable params: 13,854 (54.12 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
opt = Adam(0.005,beta_1 = 0.9, beta_2 = 0.999, decay = 0.01) 
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [27]:
m = X_train1.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(y_train1.swapaxes(0,1))

In [29]:
model.fit([X_train1, s0, c0], outputs, epochs=100, batch_size=500)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4133 - loss: 1.0823
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4133 - lo

In [34]:
predictions = model.predict([X_test1, s00, c00])
predictions = np.argmax(predictions, axis = -1)
output = [ranking[int(x)] for x in predictions]
for i in range(len(output)): 
    print(f"Comment: {X_test[i]}\nRanking: {y_test[i]}, prediction: {output[i]}\n\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Comment: I am very satisfied with this product 
Ranking: Above Average , prediction: Above Average 


Comment: I like this but the design could be better 
Ranking: Average , prediction: Above Average 


Comment: It does not work 
Ranking: Below Average , prediction: Above Average 


Comment: Too expensive 
Ranking: Below Average , prediction: Above Average 


Comment: Looks beautiful . 
Ranking: Above Average , prediction: Above Average 


Comment: love this but too expensive 
Ranking: Average , prediction: Above Average 


Comment: Works well but too large 
Ranking: Average , prediction: Above Average 


Comment: does not work for my son 
Ranking: Below Average , prediction: Above Average 


Comment: Great quality 
Ranking: Above Average , prediction: Above Average 


Comment: Works well but do not like the design 
Ranking: Average , prediction: Above Average 


Comment: I am not sure about the smell 
Ranking: Average , prediction: Above Average 



### Try mapping the inputs to the GloVe word representations to measure performance: 

In [35]:
dffed = df.iloc[:,0]
x = dffed.to_numpy()
dfrank = df.iloc[:,1]

y = dfrank.to_numpy()
print(x[1:5])
print(y[:3])

['I really like this . ' 'I like this but the design could be better '
 'I do not like the smell ' 'Works well but the smell is too strong . ']
['Above Average ' 'Above Average ' 'Average ']


In [36]:
ranking = np.unique(y)
ranking = ranking.tolist()
ranking

['Above Average ', 'Average ', 'Below Average ']

In [37]:
#dividing the dataset into 75% training set and 25% test set: 
x = x.tolist()
X_train, X_test, y_train, y_test = train_test_split(x,y, 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True) 
print(X_train[1:5])
print(y_train[1:5])

['looks amazing ', 'love this but design could be better ', 'I love the color ', 'good quality ']
['Above Average ' 'Average ' 'Above Average ' 'Average ']


In [39]:
def edit_txt(review):
    """
    This function receives a text and returns it edited as follows: 
    1, all words converted to lower case 
    2, integers removed
    3, tokenize the words 
    4, punctuation removed 
    5, common words that are unnecessary are removed. 
    """
    
    review_edited = []

    #Converting to lower case: 
    review_edited = review.lower() 
    
    #Removing integers: 
    pattern = r'[0-9]'
    # Match all digits in the string and replace them with an empty string
    review_edited = re.sub(pattern, '', review_edited) 

    #Tokenize the comment: 
    review_edited = word_tokenize(review_edited) 

    #Removing punctuation 
    tokenizer = RegexpTokenizer(r'\w+')
    review_edited = [''.join(tokenizer.tokenize(word)) for word in review_edited if len(tokenizer.tokenize(word))>0]

    #Removing common words: 
    #remove_list = stopwords.words('english') 
    #to_remove = [ "not",'don',"don't",'should',"should've", 'ain','aren',"aren't",'couldn',"couldn't",'didn',"didn't",'doesn',"doesn't",'hadn',"hadn't",'hasn',"hasn't",'haven',"haven't",'isn',"isn't",'mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',"shouldn't",'wasn',"wasn't",'weren',"weren't",'won',"won't",'wouldn', "wouldn't"]
 
    #review_edited = [word for word in review_edited if not word in remove_list]
    return(review_edited) 



In [45]:
# Edit the text in the training and texting datasets: 
X_train = [edit_txt(comment) for comment in X_train]
X_test = [edit_txt(comment) for comment in X_test]

In [49]:
# Load the word embeddigns:
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

words =  list(embeddings_dict.keys())
vectors = [embeddings_dict[word] for word in words]

In [47]:
def gvec_input(x,m,e): 

    "    This function takes any input, x, and returns a glove vector based on the \n",
    "    words introduced in the vocabulary (400,000 words). This function returns k vectors where k is the number of words in the \n",
    "    sentence. Every vector corresponds to a word in the dictionary and each entry will describe a feature of the word. \n",
    "    \n",
    "    inputs: \n",
    "    \n",
    "    x (string) : a statement from customers. \n",
    "    m (int)    : size of the sequence \n",
    "    e (int)    : size of the embeddings \n",
    "    outputs: \n",
    "    v (m,n)    : where m is the number of words in the sentence and n = 50 is the number of total features describing a word. \n",
    "\n",
    n = len(x)
    gv = np.zeros((n,m, e))

    for i in range(0, n): #looping over each comment 
        txt = x[i] #select the ith comment  
        txt = (txt[:m] if len(txt) > m else txt + ['<pad>'] * (m - len(txt))) #shorten or add extra padding
        for l in range(m): #looping over each word 
        
            # add the embedding of all ones for pads
            if txt[l] == "<pad>": 
                gv[i,l,:] = np.zeros(e) 
        
            # if a word is not is the list of Glove embeddings, then assign an array which is the average of all embeddings:  
            elif txt[l] not in words: 
                gv[i,l,:] = np.mean(vectors, axis = 0)
                # add the word embeddings: 
            else: 
                gv[i,l,:] = embeddings_dict[txt[l]]
    return(gv)

In [82]:
m = 15
e = 50 
X_trainmod = gvec_input(X_train,m,e) 
X_testmod  = gvec_input(X_test,m,e) 

In [83]:
print(X_trainmod.shape)
print(X_testmod.shape)

(75, 15, 50)
(25, 15, 50)


In [84]:
# Map the y_training and y_testing datasets to Boolean 0, 1: 
y_trainmod = (np.array([vec_output(y) for y in y_train])).reshape(len(y_train), len(ranking))
y_testmod = (np.array([vec_output(y) for y in y_test])).reshape(len(y_test),len(ranking))
y_trainmod[0:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [85]:
Tx = 15
len_dictionary = 50

model = modelf(Tx, Ty, n_a, n_s, len_dictionary, ranking)
model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 15, 50)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ s0 (InputLayer)     │ (None, 30)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_5     │ (None, 15, 20)    │      4,880 │ input_layer_5[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_5     │ (None, 15, 30)    │          0 │ s0[0][0]          │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 15, 50)    │          0 │ bidirectional_5[… │
│ (Concatenate)       │                   │            │ repeat_vector_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 15, 10)    │        510 │ concatenate_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 15, 1)     │         11 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_10          │ (None, 15, 1)     │          0 │ dense_16[0][0]    │
│ (Softmax)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_5 (Dot)         │ (None, 1, 20)     │          0 │ softmax_10[0][0], │
│                     │                   │            │ bidirectional_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ c0 (InputLayer)     │ (None, 30)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 30),      │      6,120 │ dot_5[0][0],      │
│                     │ (None, 30),       │            │ s0[0][0],         │
│                     │ (None, 30)]       │            │ c0[0][0]          │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 3)         │         93 │ lstm[5][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_11          │ (None, 3)         │          0 │ dense_17[0][0]    │
│ (Softmax)           │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,614 (45.37 KB)

 Trainable params: 11,614 (45.37 KB)

 Non-trainable params: 0 (0.00 B)

In [86]:
opt = Adam(0.005,beta_1 = 0.9, beta_2 = 0.999, decay = 0.01) 
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

In [87]:
m = X_trainmod.shape[0]
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(y_trainmod.swapaxes(0,1))

In [90]:
model.fit([X_trainmod, s0, c0], y_trainmod, epochs=100, batch_size=500)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9467 - loss: 0.3063
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9467 - loss: 0.3059
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9467 - loss: 0.3055
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9467 - loss: 0.3051
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9467 - loss: 0.3048
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9467 - loss: 0.3044
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9467 - loss: 0.3041
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9467 - loss: 0.3038
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9467 - loss: 0.3034
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9467 - loss: 0.3025
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9600 - loss: 0.2787
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9467 - lo

In [91]:
m = X_testmod.shape[0]
s00 = np.zeros((m, n_s))
c00 = np.zeros((m, n_s))

model.evaluate([X_testmod,s00,c00], y_testmod)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.9600 - loss: 0.2713


[0.27132686972618103, 0.9599999785423279]

In [92]:
predictions = model.predict([X_testmod, s00, c00])
predictions = np.argmax(predictions, axis = -1)
output = [ranking[int(x)] for x in predictions]
for i in range(len(output)): 
    print(f"Comment: {X_test[i]}\nRanking: {y_test[i]}, prediction: {output[i]}\n\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
Comment: ['i', 'am', 'very', 'satisfied', 'with', 'this', 'product']
Ranking: Above Average , prediction: Below Average 


Comment: ['i', 'like', 'this', 'but', 'the', 'design', 'could', 'be', 'better']
Ranking: Average , prediction: Average 


Comment: ['it', 'does', 'not', 'work']
Ranking: Below Average , prediction: Below Average 


Comment: ['too', 'expensive']
Ranking: Below Average , prediction: Below Average 


Comment: ['looks', 'beautiful']
Ranking: Above Average , prediction: Above Average 


Comment: ['love', 'this', 'but', 'too', 'expensive']
Ranking: Average , prediction: Average 


Comment: ['works', 'well', 'but', 'too', 'large']
Ranking: Average , prediction: Average 


Comment: ['does', 'not', 'work', 'for', 'my', 'son']
Ranking: Below Average , prediction: Below Average 


Comment: ['great', 'quality']
Ranking: Above Average , prediction: Above Average 


Comment: ['works', 'well', 'but', 'do', 'not', 'like', 'the', 'design']
Ran